In [23]:
# mlp for multi-label classification
import numpy as np
from numpy import mean
from numpy import std
import keras
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import normalize



In [18]:
# get the dataset
def get_dataset():
	df = pd.read_csv('all_states_nn2.csv')
	df.drop(['Unnamed: 0','Unnamed: 0.1','results'],axis=1,inplace=True)
 
 # Categorical boolean mask
 
	y= df[['chosen_action']]
	
	X = df.drop(['chosen_action'], axis=1)
	print(X.columns)
	print(y.columns)
	categorical_feature_mask = X.dtypes==object
	# filter categorical columns using mask and turn it into a list
	categorical_cols = X.columns[categorical_feature_mask].tolist()

	for col in categorical_cols:
		one_hot = pd.get_dummies(X[col])
		X = X.drop(col,axis = 1)
	# Join the encoded df
		X = X.join(one_hot)
 
	return X.to_numpy(), y.to_numpy()


In [19]:

# define dataset
X, y = get_dataset()
# summarize dataset shape
print(X.shape, y.shape)
# summarize first few examples
for i in range(1):
 print(X[i], y[i])

Index(['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active', 'attack(doctor, actor)',
       'attack(doctor, soldier)', 'prepare_meeting(doctor, actor)',
       'prepare_trap(doctor, soldier)', 'prepare_meeting(doctor, soldier)',
       'prepare_trap(doctor, actor)'],
      dtype='object')
Index(['chosen_action'], dtype='object')
(6322, 24) (6322, 1)
[-2 -2 -2  0  0  0  2  0  0  0  1  0 -1  0  0  1  1  0  1  0  1  1  1  1] [-1]


In [41]:

# get the model
def get_model(n_inputs=24, n_outputs=1):
 
 model = Sequential()
 model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(17, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(n_outputs, activation='softmax'))
 model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
 return model

In [33]:

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 # enumerate folds
 for train_ix, test_ix in cv.split(X):
    # prepare data
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]

    X_train_normalized = normalize(X_train, axis=0)
    X_test_normalized = normalize(X_test, axis=0)
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    callback = keras.callbacks.EarlyStopping(monitor='loss',
                              patience=10)
    model.fit(X_train_normalized, y_train, verbose=0, epochs=200, callbacks=[callback], batch_size= 32)
    # make a prediction on the test set
    yhat = model.predict(X_test_normalized)
    print(np.argmax(model.predict(X_test_normalized), axis=-1))
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    # store result
    print('>%.3f' % acc)
    results.append(acc)
 return results

In [40]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

n_inputs, n_outputs = X.shape[1], y.shape[1]
model = get_model(n_inputs, n_outputs)
estimator = KerasClassifier(build_fn=get_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\kacpe\AppData\Local\Temp\ipykernel_13012\454210977.py:7: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=get_model, epochs=200, batch_size=5, verbose=0)


Baseline: nan% (nan%)


In [42]:
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

Index(['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active', 'attack(doctor, actor)',
       'attack(doctor, soldier)', 'prepare_meeting(doctor, actor)',
       'prepare_trap(doctor, soldier)', 'prepare_meeting(doctor, soldier)',
       'prepare_trap(doctor, actor)'],
      dtype='object')
Index(['chosen_action'], dtype='object')
20/20 [==============================] - 0s 837us/step
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [13]:
from numpy import asarray
n_inputs, n_outputs = X.shape[1], y.shape[1]
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose=0, epochs=100)
# make a prediction for new data


['doctor_altruism', 'doctor_likes_soldier', 'doctor_likes_actor',
       'doctor_has_food', 'doctor_has_meds', 'soldier_altruism',
       'soldier_likes_doctor', 'soldier_likes_actor', 'soldier_has_food',
       'soldier_has_meds', 'actor_altruism', 'actor_likes_doctor',
       'actor_likes_soldier', 'actor_has_food', 'actor_has_meds',
       'food_quantity', 'meds_quantity', 'active']
['wants_meds', 'wants_food', 'attacks_doctor', 'attacks_soldier',
       'attacks_actor']

In [14]:
np.set_printoptions(suppress=True)
row = [-2, #'doctor_altruism'
        -2, #'doctor_likes_soldier'
        -2, #'doctor_likes_actor'
        0, #'doctor_has_food'
        0, #'doctor_has_meds'
        0, #soldier_altruism
        2, #'soldier_likes_doctor'
        0, #'soldier_likes_actor'
        0, #'soldier_has_food'
        0, # 'soldier_has_meds'
        1, # 'actor_altruism'
        0, #'actor_likes_doctor'
        -1, # 'actor_likes_soldier'
        0, #'actor_has_food'
        0, #'actor_has_meds'
        1, #'food_quantity'
        1, #'meds_quantity'
        0 #'active'
        ]
newX = asarray([row])
yhat = model.predict(newX)
print(np.argmax(yhat, axis=-1))
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 51ms/step
[0]
Predicted: [0.00016014]


In [15]:
np.set_printoptions(suppress=True)
row = [-2, #'doctor_altruism'
        -2, #'doctor_likes_soldier'
        -2, #'doctor_likes_actor'
        0, #'doctor_has_food'
        0, #'doctor_has_meds'
        0, #soldier_altruism
        2, #'soldier_likes_doctor'
        0, #'soldier_likes_actor'
        0, #'soldier_has_food'
        0, # 'soldier_has_meds'
        1, # 'actor_altruism'
        0, #'actor_likes_doctor'
        1, # 'actor_likes_soldier'
        0, #'actor_has_food'
        1, #'actor_has_meds'
        1, #'food_quantity'
        0, #'meds_quantity'
        0 #'active'
        ]
newX = asarray([row])
yhat = model.predict(newX)
print(np.argmax(yhat, axis=-1))
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 22ms/step
[0]
Predicted: [0.99997115]
